### P0 MIPS Code Generator Tests

#### Original Author: Emil Sekerinski, McMaster University, revised February 2021

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

#### Error "MIPS: value too large"

In [ ]:
compileString("""
program p
  const c = 100000
  var x: integer
    x := c
""", target = 'mips')

#### Error "MIPS: no structured value parameters"

In [ ]:
compileString("""
type A = 1..10 → integer
var a: A
procedure q(f: A)
  f[2] := 4
program p
  q(a)
""", target = 'mips')

#### Error "MIPS: out of registers"

In [ ]:
compileString("""
program p
  var x: integer
    x := 0×x + (1×x + (2×x + (3×x + (4×x + (5×x + (6×x + (7×x + (8×x))))))))
""", target = 'mips')

#### Error "MIPS: level!"

In [ ]:
compileString("""
procedure q()
  var x: integer
  procedure r()
    x := 5
  x := 3
program p
  writeln()
""", target = 'mips')

#### Error "MIPS: unsupported parameter type"

In [ ]:
compileString("""
var x: integer
procedure q(b: boolean)
  b := false
program p
  q(x > 7)
""", target = 'mips')

#### Input & Output
```
	.data
	.text
	.globl main
	.ent main
main:	
	.globl p
	.ent p
p:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 12
	li $v0, 5
	syscall
	sw $v0, -12($fp)
	addi $t8, $0, 3
	lw $t0, -12($fp)
	mul $t8, $t8, $t0
	sw $t8, -12($fp)
	lw $t7, -12($fp)
	sw $t7, -4($sp)
	lw $a0, -12($fp)
	li $v0, 1
	syscall
	li $v0, 11
	li $a0, '\n'
	syscall
	li $v0, 11
	li $a0, '\n'
	syscall
	lw $t5, -12($fp)
	mul $t5, $t5, 5
	sw $t5, -4($sp)
	add $a0, $t5, $0
	li $v0, 1
	syscall
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	li $v0, 10
	syscall
	.end main
```

In [ ]:
compileString("""
program p
  var x: integer
    x ← read()
    x := 3 × x
    write(x)
    writeln()
    writeln()
    write(x × 5)
""", 'write.s', target = 'mips')

#### Parameter Passing
```
	.data
z_:	.space 40
x_:	.space 4
	.text
	.globl q
	.ent q
q:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 12
	sw $a0, -12($fp)
	lw $a0, -12($fp)
	lw $a0, -12($fp)
	li $v0, 1
	syscall
	li $v0, 11
	li $a0, '\n'
	syscall
	addi $a0, $a1, 0
	lw $a0, x_
	lw $a0, x_
	li $v0, 1
	syscall
	add $a0, $a0, $0
	add $a0, $a0, $0
	li $v0, 1
	syscall
	li $v0, 11
	li $a0, '\n'
	syscall
	lw $a1, -12($fp)
	add $a0, $a1, $0
	add $a0, $a1, $0
	li $v0, 1
	syscall
	lw $a0, x_
	lw $a0, x_
	li $v0, 1
	syscall
	li $v0, 11
	li $a0, '\n'
	syscall
	add $a0, $a0, $0
	add $a0, $a0, $0
	li $v0, 1
	syscall
	lw $a0, -12($fp)
	lw $a0, -12($fp)
	li $v0, 1
	syscall
	li $v0, 11
	li $a0, '\n'
	syscall
	lw $a0, z_+12
	lw $a0, z_+12
	li $v0, 1
	syscall
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	.globl main
	.ent main
main:	
	.globl p
	.ent p
p:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 8
	addi $t3, $0, 5
	sw $t3, x_
	addi $a0, $0, 7
	lw $a1, x_
	jal, q
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	li $v0, 10
	syscall
	.end main
```

In [ ]:
compileString("""
type T = 1..10 → integer
var x: integer
var z: T
procedure q(a: integer, b: integer)
  // -4($sp) a 4($fp) -8($sp) b ($fp)
  var y: integer // -12($fp)
    y := a; write(y); writeln()           // writes 7
    a := b; write(x); write(a); writeln() // writes 5, 5
    b := y; write(b); write(x); writeln() // writes 7, 5
    write(y); writeln()                   // writes 7
    write(z[4])                           // writes 0
program p
  x := 5; q(7, x)
""", 'params.s', target = 'mips')

#### Nested Procedures
```
	.data
b_:	.space 4
	.text
	.globl main
	.ent main
main:	
	.globl r
	.ent r
r:	
	sw $fp, -8($sp)
	sw $ra, -12($sp)
	sub $fp, $sp, 4
	sub $sp, $fp, 8
	sw $0, b_
	lw $t8, 0($fp)
	add $t8, $t8, 1
	sw $t8, 0($fp)
	add $sp, $fp, 4
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	.globl q
	.ent q
q:	
	sw $fp, -8($sp)
	sw $ra, -12($sp)
	sub $fp, $sp, 4
	sub $sp, $fp, 12
	addi $t0, $0, 1
	sw $t0, 0($fp)
	addi $t7, $0, 4
	sw $t7, -12($fp)
	lw $t5, -12($fp)
	sw $t5, -4($sp)
	jal, r
	add $sp, $fp, 4
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	.globl p
	.ent p
p:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 12
	lw $t4, b_
	sw $t4, -4($sp)
	jal, q
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	li $v0, 10
	syscall
	.end main
```

In [ ]:
compileString("""
var b: boolean
program p
  var i: integer
  procedure q(z: boolean)
    var y: integer
    procedure r(x: integer)
      b := false; x := x + 1
    z := true; y := 4; r(i)
  q(b)
""", 'nested.s', target = 'mips')

#### Booleans and Conditions
```
	.data
	.text
	.globl main
	.ent main
main:	
	.globl p
	.ent p
p:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 32
	addi $t8, $0, 7
	sw $t8, -12($fp)
	addi $t0, $0, 9
	sw $t0, -16($fp)
	addi $t7, $0, 11
	sw $t7, -20($fp)
	addi $t5, $0, 1
	sw $t5, -28($fp)
	sw $0, -32($fp)
	addi $t4, $0, 1
	beq $t4, $0, L1
L2:	
	addi $t3, $0, 7
	sw $t3, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L3
L1:	
	addi $t6, $0, 9
	sw $t6, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L3:	
	beq $0, $0, L4
L5:	
	addi $t1, $0, 7
	sw $t1, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L6
L4:	
	addi $t2, $0, 9
	sw $t2, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L6:	
	lw $t8, -28($fp)
	beq $t8, $0, L7
L8:	
	addi $t0, $0, 7
	sw $t0, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L9
L7:	
	addi $t7, $0, 9
	sw $t7, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L9:	
	lw $t5, -32($fp)
	beq $t5, $0, L10
L11:	
	addi $t4, $0, 7
	sw $t4, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L12
L10:	
	addi $t3, $0, 9
	sw $t3, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L12:	
	lw $t6, -28($fp)
	bne $t6, $0, L14
L13:	
	addi $t1, $0, 7
	sw $t1, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L15
L14:	
	addi $t2, $0, 9
	sw $t2, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L15:	
	lw $t8, -32($fp)
	bne $t8, $0, L17
L16:	
	addi $t0, $0, 7
	sw $t0, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L18
L17:	
	addi $t7, $0, 9
	sw $t7, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L18:	
	lw $t5, -28($fp)
	bne $t5, $0, L20
L19:	
	lw $t4, -28($fp)
	beq $t4, $0, L21
L22:	
L20:	
	addi $t3, $0, 7
	sw $t3, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L23
L21:	
	addi $t6, $0, 9
	sw $t6, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L23:	
	lw $t1, -28($fp)
	bne $t1, $0, L25
L24:	
	lw $t2, -32($fp)
	beq $t2, $0, L26
L27:	
L25:	
	addi $t8, $0, 7
	sw $t8, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L28
L26:	
	addi $t0, $0, 9
	sw $t0, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L28:	
	lw $t7, -32($fp)
	bne $t7, $0, L30
L29:	
	lw $t5, -28($fp)
	beq $t5, $0, L31
L32:	
L30:	
	addi $t4, $0, 7
	sw $t4, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L33
L31:	
	addi $t3, $0, 9
	sw $t3, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L33:	
	lw $t6, -32($fp)
	bne $t6, $0, L35
L34:	
	lw $t1, -32($fp)
	beq $t1, $0, L36
L37:	
L35:	
	addi $t2, $0, 7
	sw $t2, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L38
L36:	
	addi $t8, $0, 9
	sw $t8, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L38:	
	lw $t0, -28($fp)
	beq $t0, $0, L39
L40:	
	lw $t7, -28($fp)
	beq $t7, $0, L41
L42:	
	addi $t5, $0, 7
	sw $t5, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L43
L41:	
L39:	
	addi $t4, $0, 9
	sw $t4, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L43:	
	lw $t3, -28($fp)
	beq $t3, $0, L44
L45:	
	lw $t6, -32($fp)
	beq $t6, $0, L46
L47:	
	addi $t1, $0, 7
	sw $t1, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L48
L46:	
L44:	
	addi $t2, $0, 9
	sw $t2, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L48:	
	lw $t8, -32($fp)
	beq $t8, $0, L49
L50:	
	lw $t0, -28($fp)
	beq $t0, $0, L51
L52:	
	addi $t7, $0, 7
	sw $t7, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L53
L51:	
L49:	
	addi $t5, $0, 9
	sw $t5, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L53:	
	lw $t4, -32($fp)
	beq $t4, $0, L54
L55:	
	lw $t3, -32($fp)
	beq $t3, $0, L56
L57:	
	addi $t6, $0, 7
	sw $t6, -4($sp)
	addi $a0, $0, 7
	li $v0, 1
	syscall
	b, L58
L56:	
L54:	
	addi $t1, $0, 9
	sw $t1, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L58:	
	li $v0, 11
	li $a0, '\n'
	syscall
	addi $t2, $0, 1
	sw $t2, -24($fp)
	lw $t8, -24($fp)
	beq $t8, $0, L59
L60:	
	addi $t0, $0, 3
	sw $t0, -4($sp)
	addi $a0, $0, 3
	li $v0, 1
	syscall
	b, L61
L59:	
	addi $t7, $0, 5
	sw $t7, -4($sp)
	addi $a0, $0, 5
	li $v0, 1
	syscall
L61:	
	sw $0, -24($fp)
	lw $t5, -24($fp)
	beq $t5, $0, L62
L63:	
	addi $t4, $0, 3
	sw $t4, -4($sp)
	addi $a0, $0, 3
	li $v0, 1
	syscall
	b, L64
L62:	
	addi $t3, $0, 5
	sw $t3, -4($sp)
	addi $a0, $0, 5
	li $v0, 1
	syscall
L64:	
	lw $t6, -12($fp)
	lw $t1, -16($fp)
	bge $t6, $t1, L65
L66:	
	addi $t2, $0, 1
	b, L67
L65:	
	addi $t2, $0, 0
L67:	
	sw $t2, -24($fp)
	lw $t8, -24($fp)
	beq $t8, $0, L68
L69:	
	lw $t0, -12($fp)
	sw $t0, -4($sp)
	lw $a0, -12($fp)
	li $v0, 1
	syscall
	b, L70
L68:	
	lw $t7, -16($fp)
	sw $t7, -4($sp)
	lw $a0, -16($fp)
	li $v0, 1
	syscall
L70:	
	lw $t5, -12($fp)
	lw $t4, -16($fp)
	bgt $t5, $t4, L72
L71:	
	lw $t3, -28($fp)
	beq $t3, $0, L73
L74:	
L72:	
	addi $t6, $0, 1
	b, L75
L73:	
	addi $t6, $0, 0
L75:	
	sw $t6, -24($fp)
	lw $t1, -24($fp)
	beq $t1, $0, L76
L77:	
	addi $t2, $0, 3
	sw $t2, -4($sp)
	addi $a0, $0, 3
	li $v0, 1
	syscall
	b, L78
L76:	
	addi $t8, $0, 5
	sw $t8, -4($sp)
	addi $a0, $0, 5
	li $v0, 1
	syscall
L78:	
	lw $t0, -12($fp)
	lw $t7, -16($fp)
	bgt $t0, $t7, L80
L79:	
	lw $t5, -32($fp)
	beq $t5, $0, L81
L82:	
L80:	
	addi $t4, $0, 1
	b, L83
L81:	
	addi $t4, $0, 0
L83:	
	sw $t4, -24($fp)
	lw $t3, -24($fp)
	beq $t3, $0, L84
L85:	
	addi $t6, $0, 3
	sw $t6, -4($sp)
	addi $a0, $0, 3
	li $v0, 1
	syscall
	b, L86
L84:	
	addi $t1, $0, 5
	sw $t1, -4($sp)
	addi $a0, $0, 5
	li $v0, 1
	syscall
L86:	
	lw $t2, -12($fp)
	lw $t8, -16($fp)
	beq $t2, $t8, L88
L87:	
	lw $t0, -12($fp)
	lw $t7, -16($fp)
	ble $t0, $t7, L89
L90:	
L88:	
	addi $t5, $0, 1
	b, L91
L89:	
	addi $t5, $0, 0
L91:	
	sw $t5, -24($fp)
	lw $t4, -24($fp)
	beq $t4, $0, L92
L93:	
	addi $t3, $0, 3
	sw $t3, -4($sp)
	addi $a0, $0, 3
	li $v0, 1
	syscall
	b, L94
L92:	
	addi $t6, $0, 5
	sw $t6, -4($sp)
	addi $a0, $0, 5
	li $v0, 1
	syscall
L94:	
	lw $t1, -12($fp)
	lw $t2, -16($fp)
	beq $t1, $t2, L96
L95:	
	lw $t8, -12($fp)
	lw $t0, -16($fp)
	bge $t8, $t0, L97
L98:	
L96:	
	addi $t7, $0, 1
	b, L99
L97:	
	addi $t7, $0, 0
L99:	
	sw $t7, -24($fp)
	lw $t5, -24($fp)
	beq $t5, $0, L100
L101:	
	addi $t4, $0, 3
	sw $t4, -4($sp)
	addi $a0, $0, 3
	li $v0, 1
	syscall
	b, L102
L100:	
	addi $t3, $0, 5
	sw $t3, -4($sp)
	addi $a0, $0, 5
	li $v0, 1
	syscall
L102:	
	lw $t6, -32($fp)
	beq $t6, $0, L103
L104:	
	lw $t1, -12($fp)
	lw $t2, -16($fp)
	blt $t1, $t2, L105
L106:	
	addi $t8, $0, 1
	b, L107
L105:	
L103:	
	addi $t8, $0, 0
L107:	
	sw $t8, -24($fp)
	lw $t0, -24($fp)
	beq $t0, $0, L108
L109:	
	addi $t7, $0, 3
	sw $t7, -4($sp)
	addi $a0, $0, 3
	li $v0, 1
	syscall
	b, L110
L108:	
	addi $t5, $0, 5
	sw $t5, -4($sp)
	addi $a0, $0, 5
	li $v0, 1
	syscall
L110:	
	li $v0, 11
	li $a0, '\n'
	syscall
L111:	
	lw $t4, -16($fp)
	addi $t3, $0, 3
	ble $t4, $t3, L112
L113:	
	lw $t3, -16($fp)
	sw $t3, -4($sp)
	lw $a0, -16($fp)
	li $v0, 1
	syscall
	lw $t6, -16($fp)
	sub $t6, $t6, 1
	sw $t6, -16($fp)
	b, L111
L112:	
	lw $t1, -16($fp)
	sw $t1, -4($sp)
	lw $a0, -16($fp)
	li $v0, 1
	syscall
	li $v0, 11
	li $a0, '\n'
	syscall
	lw $t2, -12($fp)
	lw $t8, -16($fp)
	blt $t2, $t8, L115
L114:	
	lw $t0, -28($fp)
	beq $t0, $0, L116
L117:	
	lw $t7, -12($fp)
	sw $t7, -4($sp)
	lw $a0, -12($fp)
	li $v0, 1
	syscall
L116:	
L115:	
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	li $v0, 10
	syscall
	.end main
```

In [ ]:
compileString("""
program p
  const five = 5
  const seven = 7
  const always = true
  const never = false
  var x, y, z: integer
  var b, t, f: boolean
    x := seven; y := 9; z := 11; t := true; f := false
    if true then write(7) else write(9)    // writes 7
    if false then write(7) else write(9)   // writes 9
    if t then write(7) else write(9)       // writes 7
    if f then write(7) else write(9)       // writes 9
    if ¬ t then write(7) else write(9)     // writes 9
    if ¬ f then write(7) else write(9)     // writes 7
    if t or t then write(7) else write(9)  // writes 7
    if t or f then write(7) else write(9)  // writes 7
    if f or t then write(7) else write(9)  // writes 7
    if f or f then write(7) else write(9)  // writes 9
    if t and t then write(7) else write(9) // writes 7
    if t and f then write(7) else write(9) // writes 9
    if f and t then write(7) else write(9) // writes 9
    if f and f then write(7) else write(9) // writes 9
    writeln()
    b := true
    if b then write(3) else write(5) // writes 3
    b := false
    if b then write(3) else write(5) // writes 5
    b := x < y
    if b then write(x) else write(y) // writes 7
    b := (x > y) or t
    if b then write(3) else write(5) // writes 3
    b := (x > y) or f
    if b then write(3) else write(5) // writes 5
    b := (x = y) or (x > y)
    if b then write(3) else write(5) // writes 5
    b := (x = y) or (x < y)
    if b then write(3) else write(5) // writes 3
    b := f and (x ≥ y)
    if b then write(3) else write(5) // writes 5
    writeln()
    while y > 3 do                   // writes 9, 8, 7, 6, 5, 4
      write(y); y := y - 1
    write(y); writeln()              // writes 3
    if ¬(x < y) and t then
      write(x)                       // writes 7
""", 'cond.s', target = 'mips')

#### Constant Folding, Local & Global Variables
```
	.data
y_:	.space 4
x_:	.space 4
	.text
	.globl q
	.ent q
q:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 12
	addi $t8, $0, 3
	sw $t8, -12($fp)
	addi $t0, $0, 1
	beq $t0, $0, L1
L2:	
	lw $t7, -12($fp)
	sw $t7, y_
	b, L3
L1:	
	addi $t5, $0, 7
	sw $t5, y_
L3:	
	lw $t4, y_
	sw $t4, -4($sp)
	lw $a0, y_
	li $v0, 1
	syscall
	beq $0, $0, L4
L5:	
	lw $t3, -12($fp)
	sw $t3, y_
	b, L6
L4:	
	addi $t6, $0, 7
	sw $t6, y_
L6:	
	lw $t1, y_
	sw $t1, -4($sp)
	lw $a0, y_
	li $v0, 1
	syscall
	addi $t2, $0, 1
	beq $t2, $0, L7
L8:	
	lw $t8, -12($fp)
	sw $t8, y_
	b, L9
L7:	
	addi $t0, $0, 7
	sw $t0, y_
L9:	
	lw $t7, y_
	sw $t7, -4($sp)
	lw $a0, y_
	li $v0, 1
	syscall
	beq $0, $0, L10
L11:	
	lw $t5, -12($fp)
	sw $t5, y_
	b, L12
L10:	
	addi $t4, $0, 7
	sw $t4, y_
L12:	
	lw $t3, y_
	sw $t3, -4($sp)
	lw $a0, y_
	li $v0, 1
	syscall
	beq $0, $0, L13
L14:	
	addi $t6, $0, 5
	sw $t6, -4($sp)
	addi $a0, $0, 5
	li $v0, 1
	syscall
	b, L15
L13:	
	addi $t1, $0, 9
	sw $t1, -4($sp)
	addi $a0, $0, 9
	li $v0, 1
	syscall
L15:	
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	.globl main
	.ent main
main:	
	.globl p
	.ent p
p:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 8
	addi $t2, $0, 7
	sw $t2, x_
	jal, q
	lw $t8, x_
	sw $t8, -4($sp)
	lw $a0, x_
	li $v0, 1
	syscall
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	li $v0, 10
	syscall
	.end main
```

In [ ]:
compileString("""
const seven = (9 mod 3 + 5 × 3) div 2
type int = integer
var x, y: integer
procedure q()
  const sotrue = true and true
  const sofalse = false and true
  const alsotrue = false or true
  const alsofalse = false or false
  var x: int
    x := 3
    if sotrue then y := x else y := seven
    write(y)                                       // writes 3
    if sofalse then y := x else y := seven
    write(y)                                       // writes 7
    if alsotrue then y := x else y := seven
    write(y)                                       // writes 3
    if alsofalse then y := x else y := seven
    write(y)                                       // writes 7
    if ¬(true or false) then write(5) else write(9)// writes 9
program p
  x := 7; q(); write(x)                            // writes 7
""", 'folding.s', target = 'mips')

#### Procedures
```
	.data
g_:	.space 4
	.text
	.globl q
	.ent q
q:	
	sw $fp, -8($sp)
	sw $ra, -12($sp)
	sub $fp, $sp, 4
	sub $sp, $fp, 12
	addi $t8, $0, 9
	sw $t8, -12($fp)
	lw $t0, -12($fp)
	lw $t7, 0($fp)
	ble $t0, $t7, L1
L2:	
	lw $t5, -12($fp)
	sw $t5, -4($sp)
	lw $a0, -12($fp)
	li $v0, 1
	syscall
	b, L3
L1:	
	lw $t4, g_
	sw $t4, -4($sp)
	lw $a0, g_
	li $v0, 1
	syscall
L3:	
	add $sp, $fp, 4
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	.globl main
	.ent main
main:	
	.globl p
	.ent p
p:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 8
	addi $t3, $0, 5
	sw $t3, g_
	addi $t6, $0, 7
	sw $t6, -4($sp)
	jal, q
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	li $v0, 10
	syscall
	.end main
```

In [ ]:
compileString("""
var g: integer          // global variable
procedure q(v: integer) // value parameter
  var l: integer        // local variable
    l := 9
    if l > v then write(l)
    else write(g)
program p
  g := 5; q(7)          // writes 9
""", 'proc.s', target = 'mips')

#### Illustrating Lack of Optimization
```
	.data
	.text
	.globl main
	.ent main
main:	
	.globl p
	.ent p
p:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 12
	addi $t8, $0, 5
	sw $t8, -12($fp)
	lw $t0, -12($fp)
	add $t0, $t0, 0
	sw $t0, -12($fp)
	lw $t5, -12($fp)
	add $t7, $0, $t5
	sw $t7, -12($fp)
	lw $t4, -12($fp)
	mul $t4, $t4, 1
	sw $t4, -12($fp)
	addi $t3, $0, 1
	lw $t6, -12($fp)
	mul $t3, $t3, $t6
	sw $t3, -12($fp)
	lw $t1, -12($fp)
	add $t1, $t1, 3
	sw $t1, -12($fp)
	addi $t2, $0, 3
	lw $t8, -12($fp)
	add $t2, $t2, $t8
	sw $t2, -12($fp)
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	li $v0, 10
	syscall
	.end main
```

In [ ]:
compileString("""
program p
  var x: integer
    x := 5
    x := x + 0
    x := 0 + x
    x := x × 1
    x := 1 × x
    x := x + 3
    x := 3 + x
""", 'opt.s', target = 'mips')

#### Two-dimensional Array
```
	.data
b_:	.space 4
y_:	.space 4
x_:	.space 308
	.text
	.globl main
	.ent main
main:	
	.globl p
	.ent p
p:	
	sw $fp, -4($sp)
	sw $ra, -8($sp)
	sub $fp, $sp, 0
	sub $sp, $fp, 8
	lw $t8, y_
	sub $t8, $t8, 3
	mul $t8, $t8, 44
	sw $0, x_+16($t8)
	lw $t0, y_
	sub $t0, $t0, 3
	mul $t0, $t0, 44
	lw $t7, y_
	add $t7, $t7, 1
	sub $t7, $t7, 1
	mul $t7, $t7, 4
	add $t7, $t0, $t7
	lw $t5, x_($t7)
	sw $t5, b_
	add $sp, $fp, 0
	lw $ra, -8($fp)
	lw $fp, -4($fp)
	jr $ra
	li $v0, 10
	syscall
	.end main
```

In [ ]:
compileString("""
type R = boolean
type S = 1..11 → R
type T = 3..9 → S
var x: T
var y: integer
var b: boolean
program p
  x[y][5] := false
  b := x[y][y + 1]
""", 'twoD.s', target = 'mips')